In [3]:
import re

import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df_fnc = pd.read_csv('../input/trends-assessment-prediction/fnc.csv')
df_loading = pd.read_csv('../input/trends-assessment-prediction/loading.csv')
df_train_scores = pd.read_csv('../input/trends-assessment-prediction/train_scores.csv')

fnc_features, loading_features = list(df_fnc.columns[1:]), list(df_loading.columns[1:])
df_train_scores['is_train'] = 1
df = df_fnc.merge(df_loading, on='Id')
df = df.merge(df_train_scores, how='left', on='Id')

df.loc[df['is_train'].isnull(), 'is_train'] = 0
df['is_train'] = df['is_train'].astype(np.uint8)

print(f'Static FNC Correlation Shape = {df_fnc.shape}')
print(f'Static FNC Correlation Memory Usage = {df_fnc.memory_usage().sum() / 1024 ** 2:.2f} MB')
print(f'sMRI SBM Loadings Shape = {df_loading.shape}')
print(f'sMRI SBM Loadings Memory Usage = {df_loading.memory_usage().sum() / 1024 ** 2:.2f} MB')
print(f'Train Scores Shape = {df_train_scores.shape}')
print(f'Train Scores Memory Usage = {df_train_scores.memory_usage().sum() / 1024 ** 2:.2f} MB')
print('-------------------------------------')
print(f'Train & Test Set Shape = {df.shape}')
print(f'Train & Test Set Memory Usage = {df.memory_usage().sum() / 1024 ** 2:.2f} MB')

del df_fnc, df_loading, df_train_scores


FileNotFoundError: [Errno 2] File ../input/trends-assessment-prediction/fnc.csv does not exist: '../input/trends-assessment-prediction/fnc.csv'

## **1. Train Scores (Targets)**
`train_scores.csv` is the file which consists of target features. Those features are `age`, `domain1_var1`, `domain1_var2`, `domain2_var1` and `domain2_var2`. There are 5 target features to predict and submissions are scored using feature-weighted, normalized absolute errors.

**score** ${= \Large \sum\limits_{f} w_{f} \Big( \frac{\sum_{i} \big| y_{f,i}  - \hat{y}_{f,i}\big| }{\sum_{i} \hat{y}_{f,i}} \Big) }$

The weights are `[.3, .175, .175, .175, .175]` corresponding to features `[age, domain1_var1, domain1_var2, domain2_var1, domain2_var2]`. This means every targets normalized absolute error is independent from each other. They can be trained and predicted with a single model or 5 different models.

Even though some of target features are slightly tailed, all of them follow a normal distribution. Their descriptive statistical summary are very similar to each other.

A small percentage of values are missing in target features except `age`. Target features in the same domain have same number of missing values and they are missing in same samples.  Those are skipped in the score calculation. However, every row should be predicted in the submission file.

In [ ]:
def plot_target(target_feature):    
    
    if target_feature == 'age':
        print(f'Target feature {target_feature} Statistical Analysis\n{"-" * 39}')
    else:
        print(f'Target feature {target_feature} Statistical Analysis\n{"-" * 48}')
        
    print(f'Mean: {df[target_feature].mean():.4}  -  Median: {df[target_feature].median():.4}  -  Std: {df[target_feature].std():.4}')
    print(f'Min: {df[target_feature].min():.4}  -  25%: {df[target_feature].quantile(0.25):.4}  -  50%: {df[target_feature].quantile(0.5):.4}  -  75%: {df[target_feature].quantile(0.75):.4}  -  Max: {df[target_feature].max():.4}')
    print(f'Skew: {df[target_feature].skew():.4}  -  Kurtosis: {df[target_feature].kurtosis():.4}')
    missing_values_count = df[(df['is_train'] == 1) & (df[target_feature]).isnull()].shape[0]
    training_samples_count = df[df['is_train'] == 1].shape[0]
    print(f'Missing Values: {missing_values_count}/{training_samples_count} ({missing_values_count * 100 / training_samples_count:.4}%)')

    fig = plt.subplots(figsize=(6, 4), dpi=100)

    sns.distplot(df[target_feature], label=target_feature)

    plt.xlabel('')
    plt.tick_params(axis='x', labelsize=12)
    plt.tick_params(axis='y', labelsize=12)
    plt.title(f'{target_feature} Distribution in Training Set')
    plt.show()
    
for target_feature in ['age', 'domain1_var1', 'domain1_var2', 'domain2_var1', 'domain2_var2']:
    plot_target(target_feature)

Target features are not correlated with each other too much. The strongest correlations are between `age` and `domain1_var1` (**0.34**), and between  `age` and `domain2_var1` (**0.23**). There might be a relationship between  `age` and var1 features.

In [ ]:
fig = plt.figure(figsize=(10, 10), dpi=100)

sns.heatmap(df[['age', 'domain1_var1', 'domain1_var2', 'domain2_var1', 'domain2_var2']].corr(),
            annot=True,
            square=True,
            cmap='coolwarm',
            annot_kws={'size': 14}, 
            fmt='.2f')   

plt.tick_params(axis='x', labelsize=14, rotation=45)
plt.tick_params(axis='y', labelsize=14, rotation=45)
    
plt.title('Target Features Correlations', size=18, pad=18)
plt.show()

## **2. sMRI SBM Loadings**
The first set of features are source-based morphometry (SBM) loadings. These are subject-level weights from a group-level ICA decomposition of gray matter concentration maps from structural MRI (sMRI) scans. Those features are for both training and test samples.

There are **26** features in `loading.csv` file without `Id`. Those features are named from `IC_01` to `IC_30`, but `IC_19`, `IC_23`, `IC_25` and `IC_27` don't exist. They are parts of brain and their explanations are listed below:

`
IC_01 - Cerebellum
IC_02 - ACC+mpfc
IC_03 - Caudate
IC_04 - Cerebellum
IC_05 - Calcarine
IC_06 - Calcarine
IC_07 - Precuneus+PCC
IC_08 - Frontal
IC_09 - IPL+AG
IC_10 - MTG
IC_11 - Frontal
IC_12 - SMA
IC_13 - Temporal Pole
IC_14 - Temporal Pole + Fusiform
IC_15 - STG
IC_16 - Middle Occipital?
IC_17 - Cerebellum
IC_18 - Cerebellum
IC_20 - MCC
IC_21 - Temporal Pole + Cerebellum
IC_22 - Insula + Caudate
IC_24 - IPL+Postcentral
IC_26 - Inf+Mid Frontal
IC_28 - Calcarine
IC_29 - MTG
IC_30 - Inf Frontal
`

All of the loading features follow a normal distribution. Their distributions and descriptive statistical summary in training and test samples are very similar except `IC_20`. It is an exception because the distribution of `IC_20` in test samples is slightly shifted. This feature may require some preprocessing.

None of the loading features have a visible relationship with any of the targets. Data points of loading features are scattered around the means of `domain1_var1`, `domain1_var2`, `domain2_var1`, `domain2_var2`, however `age` has a tiny relationship with loading features. This relationship is not easy to detect but it looks like `age` is easier to predict than other target features.

In [ ]:
def plot_loading(loading_feature):    
    
    print(f'Loading feature {loading_feature} Statistical Analysis\n{"-" * 42}')
        
    print(f'Mean: {df[loading_feature].mean():.4}  -  Median: {df[loading_feature].median():.4}  -  Std: {df[loading_feature].std():.4}')
    print(f'Min: {df[loading_feature].min():.4}  -  25%: {df[loading_feature].quantile(0.25):.4}  -  50%: {df[loading_feature].quantile(0.5):.4}  -  75%: {df[loading_feature].quantile(0.75):.4}  -  Max: {df[loading_feature].max():.4}')
    print(f'Skew: {df[loading_feature].skew():.4}  -  Kurtosis: {df[loading_feature].kurtosis():.4}')
    missing_values_count = df[df[loading_feature].isnull()].shape[0]
    training_samples_count = df.shape[0]
    print(f'Missing Values: {missing_values_count}/{training_samples_count} ({missing_values_count * 100 / training_samples_count:.4}%)')

    fig, axes = plt.subplots(ncols=3, nrows=2, figsize=(25, 12), dpi=100, constrained_layout=True)
    title_size = 18
    label_size = 18

    # Loading Feature Training and Test Set Distribution
    sns.distplot(df[df['is_train'] == 1][loading_feature], label='Training', ax=axes[0][0])
    sns.distplot(df[df['is_train'] == 0][loading_feature], label='Test', ax=axes[0][0])
    axes[0][0].set_xlabel('')
    axes[0][0].tick_params(axis='x', labelsize=label_size)
    axes[0][0].tick_params(axis='y', labelsize=label_size)
    axes[0][0].legend()
    axes[0][0].set_title(f'{loading_feature} Distribution in Training and Test Set', size=title_size, pad=title_size)
    
    # Loading Feature vs age
    sns.scatterplot(df[loading_feature], df['age'], ax=axes[0][1])
    axes[0][1].set_title(f'{loading_feature} vs age', size=title_size, pad=title_size)
    axes[0][1].set_xlabel('')
    axes[0][1].set_ylabel('')
    axes[0][1].tick_params(axis='x', labelsize=label_size)
    axes[0][1].tick_params(axis='y', labelsize=label_size)
    
    # Loading Feature vs domain1_var1
    sns.scatterplot(df[loading_feature], df['domain1_var1'], ax=axes[0][2])
    axes[0][2].set_title(f'{loading_feature} vs domain1_var1', size=title_size, pad=title_size)
    axes[0][2].set_xlabel('')
    axes[0][2].set_ylabel('')
    axes[0][2].tick_params(axis='x', labelsize=label_size)
    axes[0][2].tick_params(axis='y', labelsize=label_size)
    
    # Loading Feature vs domain1_var2
    sns.scatterplot(df[loading_feature], df['domain1_var2'], ax=axes[1][0])
    axes[1][0].set_title(f'{loading_feature} vs domain1_var2', size=title_size, pad=title_size)
    axes[1][0].set_xlabel('')
    axes[1][0].set_ylabel('')
    axes[1][0].tick_params(axis='x', labelsize=label_size)
    axes[1][0].tick_params(axis='y', labelsize=label_size)
    
    # Loading Feature vs domain2_var1
    sns.scatterplot(df[loading_feature], df['domain2_var1'], ax=axes[1][1])
    axes[1][1].set_title(f'{loading_feature} vs domain2_var1', size=title_size, pad=title_size)
    axes[1][1].set_xlabel('')
    axes[1][1].set_ylabel('')
    axes[1][1].tick_params(axis='x', labelsize=label_size)
    axes[1][1].tick_params(axis='y', labelsize=label_size)
    
    # Loading Feature vs domain2_var2
    sns.scatterplot(df[loading_feature], df['domain2_var2'], ax=axes[1][2])
    axes[1][2].set_title(f'{loading_feature} vs domain2_var2', size=title_size, pad=title_size)
    axes[1][2].set_xlabel('')
    axes[1][2].set_ylabel('')
    axes[1][2].tick_params(axis='x', labelsize=label_size)
    axes[1][2].tick_params(axis='y', labelsize=label_size)
    
    plt.show()
    
for loading_feature in sorted(loading_features):
    plot_loading(loading_feature)


There are strong correlations between `age` and some loading features that exceed **-0.4**, but none of the loading features are correlated with other targets.

Loading features have decent correlations between themselves that exceed **0.5** and **-0.5**. Some of those high positive correlations belong to feature groups which are from the same part of the brain. However, all features from the same part of the brain are not necessarily correlated with each other, so there is no pattern here.

In [ ]:
loading_target_features = sorted(loading_features) + ['age', 'domain1_var1', 'domain1_var2', 'domain2_var1', 'domain2_var2']

fig = plt.figure(figsize=(30, 30), dpi=100)

sns.heatmap(df[loading_target_features].corr(),
            annot=True,
            square=True,
            cmap='coolwarm',
            annot_kws={'size': 15}, 
            fmt='.2f')   

plt.tick_params(axis='x', labelsize=18, rotation=45)
plt.tick_params(axis='y', labelsize=18, rotation=45)

plt.title('Target and Loading Features Correlations', size=25, pad=25)
plt.show()

## **3. Static FNC Correlation Features**
The second set of features are static functional network connectivity (FNC) matrices. These are the subject-level cross-correlation values among 53 component timecourses estimated from GIG-ICA of resting state functional MRI (fMRI).

There are **1378** features in `fnc.csv` file without `Id`. Those features are named as `Network1(X)_vs_Network2(Y)` and there are **6** different networks. Network names and abbreviations are listed below:

`
SCN - Sub-cortical Network
ADN - Auditory Network
SMN - Sensorimotor Network
VSN - Visual Network
CON - Cognitive-control Network    
DMN - Default-mode Network
CBN - Cerebellar Network
`

Those groups might be useful to analyze **1378** features part by part.

In [ ]:
def plot_fnc(fnc_feature):    
    
    print(f'FNC feature {fnc_feature} Statistical Analysis\n{"-" * 51}')
        
    print(f'Mean: {df[fnc_feature].mean():.4}  -  Median: {df[fnc_feature].median():.4}  -  Std: {df[fnc_feature].std():.4}')
    print(f'Min: {df[fnc_feature].min():.4}  -  25%: {df[fnc_feature].quantile(0.25):.4}  -  50%: {df[fnc_feature].quantile(0.5):.4}  -  75%: {df[fnc_feature].quantile(0.75):.4}  -  Max: {df[fnc_feature].max():.4}')
    print(f'Skew: {df[fnc_feature].skew():.4}  -  Kurtosis: {df[fnc_feature].kurtosis():.4}')
    missing_values_count = df[df[fnc_feature].isnull()].shape[0]
    training_samples_count = df.shape[0]
    print(f'Missing Values: {missing_values_count}/{training_samples_count} ({missing_values_count * 100 / training_samples_count:.4}%)')

    fig, axes = plt.subplots(ncols=3, nrows=2, figsize=(25, 12), dpi=100, constrained_layout=True)
    title_size = 18
    label_size = 18

    # FNC Feature Training and Test Set Distribution
    sns.distplot(df[df['is_train'] == 1][fnc_feature], label='Training', ax=axes[0][0])
    sns.distplot(df[df['is_train'] == 0][fnc_feature], label='Test', ax=axes[0][0])
    axes[0][0].set_xlabel('')
    axes[0][0].tick_params(axis='x', labelsize=label_size)
    axes[0][0].tick_params(axis='y', labelsize=label_size)
    axes[0][0].legend()
    axes[0][0].set_title(f'{fnc_feature} Distribution in Training and Test Set', size=title_size, pad=title_size)
    
    # FNC Feature vs age
    sns.scatterplot(df[fnc_feature], df['age'], ax=axes[0][1])
    axes[0][1].set_title(f'{fnc_feature} vs age', size=title_size, pad=title_size)
    axes[0][1].set_xlabel('')
    axes[0][1].set_ylabel('')
    axes[0][1].tick_params(axis='x', labelsize=label_size)
    axes[0][1].tick_params(axis='y', labelsize=label_size)
    
    # FNC Feature vs domain1_var1
    sns.scatterplot(df[fnc_feature], df['domain1_var1'], ax=axes[0][2])
    axes[0][2].set_title(f'{fnc_feature} vs domain1_var1', size=title_size, pad=title_size)
    axes[0][2].set_xlabel('')
    axes[0][2].set_ylabel('')
    axes[0][2].tick_params(axis='x', labelsize=label_size)
    axes[0][2].tick_params(axis='y', labelsize=label_size)
    
    # FNC Feature vs domain1_var2
    sns.scatterplot(df[fnc_feature], df['domain1_var2'], ax=axes[1][0])
    axes[1][0].set_title(f'{fnc_feature} vs domain1_var2', size=title_size, pad=title_size)
    axes[1][0].set_xlabel('')
    axes[1][0].set_ylabel('')
    axes[1][0].tick_params(axis='x', labelsize=label_size)
    axes[1][0].tick_params(axis='y', labelsize=label_size)
    
    # FNC Feature vs domain2_var1
    sns.scatterplot(df[fnc_feature], df['domain2_var1'], ax=axes[1][1])
    axes[1][1].set_title(f'{fnc_feature} vs domain2_var1', size=title_size, pad=title_size)
    axes[1][1].set_xlabel('')
    axes[1][1].set_ylabel('')
    axes[1][1].tick_params(axis='x', labelsize=label_size)
    axes[1][1].tick_params(axis='y', labelsize=label_size)
    
    # FNC Feature vs domain2_var2
    sns.scatterplot(df[fnc_feature], df['domain2_var2'], ax=axes[1][2])
    axes[1][2].set_title(f'{fnc_feature} vs domain2_var2', size=title_size, pad=title_size)
    axes[1][2].set_xlabel('')
    axes[1][2].set_ylabel('')
    axes[1][2].tick_params(axis='x', labelsize=label_size)
    axes[1][2].tick_params(axis='y', labelsize=label_size)
    
    plt.show()

### **3.1. Sub-cortical Network (SCN) Features**

First FNC feature sub-group is SCN features. This is the smallest sub-group of fnc features and there are **10** features in it. This sub-group has cross-correlations with only itself.

All of the SCN features follow a normal distribution but they are slightly tailed. Their distributions and descriptive statistical summary in training and test samples are very similar, but some of the features have higher peaks in test samples.

None of the SCN features have a visible relationship with any of the targets. Data points of SCN features are scattered around the means of `domain1_var1`, `domain1_var2`, `domain2_var1`, `domain2_var2`, however `age` has a tiny relationship with SCN features.

Some data points are very far away from mean clusters, they might be outliers.

In [ ]:
scn_pattern = r'SCN\(\d+\)_vs_[A-Z]+\(\d+\)'
scn_features = [col for col in fnc_features if re.match(scn_pattern, col)]

for scn_feature in sorted(scn_features):
    plot_fnc(scn_feature)

SCN features are strongly correlated with each other, but none of the SCN features are correlated with targets. Correlation between target and SCN features is even weaker than previous correlations.

In [ ]:
scn_target_features = sorted(scn_features) + ['age', 'domain1_var1', 'domain1_var2', 'domain2_var1', 'domain2_var2']

fig = plt.figure(figsize=(25, 25), dpi=100)

sns.heatmap(df[scn_target_features].corr(),
            annot=True,
            square=True,
            cmap='coolwarm',
            annot_kws={'size': 15}, 
            fmt='.2f')   

plt.tick_params(axis='x', labelsize=18, rotation=75)
plt.tick_params(axis='y', labelsize=18, rotation=0)

plt.title('Target and SCN Features Correlations', size=25, pad=25)
plt.show()

### **3.2. Auditory Network (ADN) Features**

Second FNC feature sub-group is ADN features. This is also the second smallest sub-group of fnc features and there are **11** features in it. This sub-group has cross-correlations with SCN (10) and itself (1).

All of the ADN features follow a normal distribution but they are slightly tailed. Their distributions and descriptive statistical summary in training and test samples are very similar, but some of the features have higher peaks in test samples.

None of the ADN features have a visible relationship with any of the targets. Data points of ADN features are scattered around the means of `domain1_var1`, `domain1_var2`, `domain2_var1`, `domain2_var2` and `age`.

In features with tailed distributions, some data points are very far away from mean clusters, they might be outliers.

In [ ]:
adn_pattern = r'ADN\(\d+\)_vs_[A-Z]+\(\d+\)'
adn_features = [col for col in fnc_features if re.match(adn_pattern, col)]

for adn_feature in sorted(adn_features):
    plot_fnc(adn_feature)

ADN features are strongly correlated with each other if only ADN is cross-correlated with SCN. There are **10** ADN(2) vs SCN(5) features and they have very strong correlation. There is only one ADN vs ADN (`ADN(56)_vs_ADN(21)`) feature and it doesn't have any significant correlation with anything. Correlations between target and ADN features is very weak which can be seen at the bottom and right end.

In [ ]:
adn_target_features = sorted(adn_features) + ['age', 'domain1_var1', 'domain1_var2', 'domain2_var1', 'domain2_var2']

fig = plt.figure(figsize=(25, 25), dpi=100)

sns.heatmap(df[adn_target_features].corr(),
            annot=True,
            square=True,
            cmap='coolwarm',
            annot_kws={'size': 15}, 
            fmt='.2f')   

plt.tick_params(axis='x', labelsize=18, rotation=75)
plt.tick_params(axis='y', labelsize=18, rotation=0)

plt.title('Target and ADN Features Correlations', size=25, pad=25)
plt.show()

### **3.3. Sensorimotor Network (SMN) Features**

Third FNC feature sub-group is SMN features. This is a large sub-group compared to previous ones and there are **99** features in it. This sub-group has cross-correlations with SCN (45), ADN (18) and itself (36).

All of the SMN features follow a normal distribution even though their shapes and tails vary a lot. Their distributions and descriptive statistical summary in training and test samples are very similar, but some of the features have small discrepancies.

None of the SMN features have a visible relationship with any of the targets. Data points of SMN features are scattered around the means of `domain1_var1`, `domain1_var2`, `domain2_var1`, `domain2_var2` and `age`.

In features with tailed distributions, some data points are very far away from mean clusters, they might be outliers.

In [ ]:
smn_pattern = r'SMN\(\d+\)_vs_[A-Z]+\(\d+\)'
smn_features = [col for col in fnc_features if re.match(smn_pattern, col)]

for smn_feature in sorted(smn_features):
    plot_fnc(smn_feature)

It is hard to identify correlations at feature level on this scale but it still gives lots of information. Since the feature names are sorted by alphabetical order, the correlations of feature groups are easy to detect.

Every cross-correlation of SMN have very strong correlations inside the second network groups. This explains the bright red blocks near the diagonal axis.

* `SMN(X)_vs_ADN(Y)`: `X` is strongly correlated with every different `Y` value for `ADN` 
* `SMN(X)_vs_SCN(Y)`: `X` is strongly correlated with every different `Y` value for `SCN` 
* `SMN(X)_vs_SMN(Y)`: `X` is strongly correlated with every different `Y` value for `SMN`

Every cross-correlation of SMN have moderate correlations inside the first network groups. This explains the repeating skin and blue color blocks along the vertical and horizontal axis.

* `SMN(X)_vs_ADN(Y)`: `Y` is strongly correlated with every different `X` value for `ADN` 
* `SMN(X)_vs_SCN(Y)`: `Y` is strongly correlated with every different `X` value for `SCN` 
* `SMN(X)_vs_SMN(Y)`: `Y` is strongly correlated with every different `X` value for `SMN`

Correlations between target and SMN features is very weak which can be seen at the bottom and right end.

In [ ]:
smn_target_features = sorted(smn_features) + ['age', 'domain1_var1', 'domain1_var2', 'domain2_var1', 'domain2_var2']

fig = plt.figure(figsize=(40, 40), dpi=100)

sns.heatmap(df[smn_target_features].corr(),
            annot=False,
            square=True,
            cmap='coolwarm',
            yticklabels=False,
            xticklabels=False)   

plt.tick_params(axis='x', labelsize=20, rotation=90)
plt.tick_params(axis='y', labelsize=20, rotation=0)

plt.title('Target and SMN Features Correlations', size=50, pad=50)
plt.show()

### **3.4. Visual Network (VSN) Features** 

Fourth FNC feature sub-group is VSN features. This is a very large sub-group compared to previous ones and there are **180** features in it. This sub-group has cross-correlations with SCN (45), ADN (18), SMN (81) and itself (36).

All of the VSN features follow a normal distribution even though their shapes and tails vary a lot. Some of the features in this group have quite long tails in both sides. Their distributions and descriptive statistical summary in training and test samples are very similar, but some of the features have small discrepancies.

Some of the VSN features have barely visible negative relationship with targets. Data points of VSN features are even more scattered around the means of `domain1_var1`, `domain1_var2`, `domain2_var1`, `domain2_var2` and `age`.

In features with heavy-tailed distributions, some data points are very far away from mean clusters, they might be outliers.

In [ ]:
vsn_pattern = r'VSN\(\d+\)_vs_[A-Z]+\(\d+\)'
vsn_features = [col for col in fnc_features if re.match(vsn_pattern, col)]

for vsn_feature in sorted(vsn_features):
    plot_fnc(vsn_feature)

It is hard to identify correlations at feature level on this scale but it still gives lots of information. Since the feature names are sorted by alphabetical order, the correlations of feature groups are easy to detect. The same pattern from SMN features can be seen on VSN features as well.

Every cross-correlation of VSN have very strong correlations inside the second network groups. This explains the bright red blocks near the diagonal axis.

* `VSN(X)_vs_ADN(Y)`: `X` is strongly correlated with every different `Y` value for `ADN` 
* `VSN(X)_vs_SCN(Y)`: `X` is strongly correlated with every different `Y` value for `SCN` 
* `VSN(X)_vs_SMN(Y)`: `X` is strongly correlated with every different `Y` value for `SMN`
* `VSN(X)_vs_VSN(Y)`: `X` is strongly correlated with every different `Y` value for `VSN`

Every cross-correlation of VSN have moderate correlations inside the first network groups. This explains the repeating skin and blue color blocks along the vertical and horizontal axis.

* `VSN(X)_vs_ADN(Y)`: `Y` is strongly correlated with every different `X` value for `ADN` 
* `VSN(X)_vs_SCN(Y)`: `Y` is strongly correlated with every different `X` value for `SCN` 
* `VSN(X)_vs_SMN(Y)`: `Y` is strongly correlated with every different `X` value for `SMN`
* `VSN(X)_vs_VSN(Y)`: `Y` is strongly correlated with every different `X` value for `VSN`

Correlations between target and VSN features is very weak which can be seen at the bottom and right end.

In [ ]:
vsn_target_features = sorted(vsn_features) + ['age', 'domain1_var1', 'domain1_var2', 'domain2_var1', 'domain2_var2']

fig = plt.figure(figsize=(40, 40), dpi=100)

sns.heatmap(df[vsn_target_features].corr(),
            annot=False,
            square=True,
            cmap='coolwarm',
            yticklabels=False,
            xticklabels=False)   

plt.tick_params(axis='x', labelsize=20, rotation=90)
plt.tick_params(axis='y', labelsize=20, rotation=0)

plt.title('Target and VSN Features Correlations', size=50, pad=50)
plt.show()

### **3.5. Cognitive-Control Network (CON) Features** 

In [ ]:
con_pattern = r'CON\(\d+\)_vs_[A-Z]+\(\d+\)'
con_features = [col for col in fnc_features if re.match(con_pattern, col)]

for con_feature in sorted(con_features):
    plot_fnc(con_feature)

In [ ]:
con_target_features = sorted(con_features) + ['age', 'domain1_var1', 'domain1_var2', 'domain2_var1', 'domain2_var2']

fig = plt.figure(figsize=(50, 50), dpi=100)

sns.heatmap(df[con_target_features].corr(),
            annot=False,
            square=True,
            cmap='coolwarm',
            yticklabels=False,
            xticklabels=False)   

plt.tick_params(axis='x', labelsize=20, rotation=90)
plt.tick_params(axis='y', labelsize=20, rotation=0)

plt.title('Target and CON Features Correlations', size=50, pad=50)
plt.show()

In [ ]:
# To Be Continued